In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, accuracy_score

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Custom Dataset class
class TORCSDataset(Dataset):
    def __init__(self, csv_file, sensor_cols, continuous_cols, discrete_cols):
        self.data = pd.read_csv(csv_file)
        self.sensor_cols = [col for col in self.data.columns if col in sensor_cols]
        # self.sensor_cols = sensor_cols
        self.continuous_cols = [col for col in self.data.columns if col in continuous_cols]
        # self.continuous_cols = continuous_cols
        self.discrete_cols = [col for col in self.data.columns if col in discrete_cols]
        # self.discrete_cols = discrete_cols

        self.features = torch.tensor(self.data[self.sensor_cols].values, dtype=torch.float32)
        self.steering_labels = torch.tensor(self.data['Steering'].values, dtype=torch.float32).view(-1, 1)
        self.accel_labels = torch.tensor(self.data['Acceleration'].values, dtype=torch.long)
        self.brake_labels = torch.tensor(self.data['Braking'].values, dtype=torch.long)
        self.gear_labels = torch.tensor(self.data['Gear'].values, dtype=torch.long)
        self.clutch_labels = torch.tensor(self.data['Clutch'].values, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return (
            self.features[idx],
            self.steering_labels[idx],
            self.accel_labels[idx],
            self.brake_labels[idx],
            self.gear_labels[idx],
            self.clutch_labels[idx]
        )

In [ ]:
# Neural Network Model
class TORCSNet(nn.Module):
    def __init__(self, input_size, num_accel_classes, num_brake_classes, num_gear_classes, num_clutch_classes=2):
        super(TORCSNet, self).__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),        
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.steering_head = nn.Linear(128, 1)  # Steering (continuous, [-1, 1])
        self.accel_head = nn.Linear(128, num_accel_classes)  # Acceleration (discrete)
        self.brake_head = nn.Linear(128, num_brake_classes)  # Braking (discrete)
        self.gear_head = nn.Linear(128, num_gear_classes)  # Gear (discrete)
        self.clutch_head = nn.Linear(128, num_clutch_classes)  # Clutch (discrete)
        
    def forward(self, x):
        shared = self.shared(x)
        steering_out = torch.tanh(self.steering_head(shared))  # [-1, 1]
        accel_out = self.accel_head(shared)  # Logits
        brake_out = self.brake_head(shared)  # Logits
        gear_out = self.gear_head(shared)  # Logits
        clutch_out = self.clutch_head(shared)  # Logits
        return steering_out, accel_out, brake_out, gear_out, clutch_out

In [ ]:
# Define columns (update based on preprocessing output)
sensor_cols = [
        'Angle', 'DistanceCovered', 'LastLapTime', 'RPM',
        'SpeedX', 'SpeedY', 'SpeedZ', 'Track_1', 'Track_2', 'Track_3', 
        'Track_4', 'Track_5', 'Track_6', 'Track_7', 'Track_8', 'Track_9',
        'Track_10', 'Track_11', 'Track_12', 'Track_13', 'Track_14', 
        'Track_15', 'Track_16', 'Track_17', 'Track_18', 'Track_19',
        'TrackPosition', 'WheelSpinVelocity_1', 'WheelSpinVelocity_2', 
        'WheelSpinVelocity_3', 'WheelSpinVelocity_4', 'Z'
]  # Placeholder: Update with actual 27 sensors
continuous_cols = ['Steering']
discrete_cols = ['Acceleration', 'Braking', 'Gear', 'Clutch']
    
    # Load datasets
train_dataset = TORCSDataset('../data_generation/data/dirt-2/train_data.csv', sensor_cols, continuous_cols, discrete_cols)
val_dataset = TORCSDataset('../data_generation/data/dirt-2/val_data.csv', sensor_cols, continuous_cols, discrete_cols)
    
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
    
# Model parameters
input_size = len(sensor_cols)
print(input_size)
print(len(train_dataset.sensor_cols))

In [ ]:
# Training function
def train_model(model, train_loader, val_loader, criterion_steering, criterion_accel, criterion_brake, criterion_gear, criterion_clutch, optimizer, num_epochs, device):
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0.0
        accel_correct = 0
        brake_correct = 0
        gear_correct = 0
        clutch_correct = 0
        total = 0
        
        for features, steering_labels, accel_labels, brake_labels, gear_labels, clutch_labels in train_loader:
            features = features.to(device)
            steering_labels = steering_labels.to(device)
            accel_labels, brake_labels = accel_labels.to(device), brake_labels.to(device)
            gear_labels, clutch_labels = gear_labels.to(device), clutch_labels.to(device)
            
            optimizer.zero_grad()
            steering_out, accel_out, brake_out, gear_out, clutch_out = model(features)
            
            loss_steering = criterion_steering(steering_out, steering_labels)
            loss_accel = criterion_accel(accel_out, accel_labels)
            loss_brake = criterion_brake(brake_out, brake_labels)
            loss_gear = criterion_gear(gear_out, gear_labels)
            loss_clutch = criterion_clutch(clutch_out, clutch_labels)
            loss = loss_steering + loss_accel + loss_brake + loss_gear + loss_clutch
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * features.size(0)
            accel_correct += (torch.argmax(accel_out, dim=1) == accel_labels).sum().item()
            brake_correct += (torch.argmax(brake_out, dim=1) == brake_labels).sum().item()
            gear_correct += (torch.argmax(gear_out, dim=1) == gear_labels).sum().item()
            clutch_correct += (torch.argmax(clutch_out, dim=1) == clutch_labels).sum().item()
            total += gear_labels.size(0)
        
        train_loss /= len(train_loader.dataset)
        accel_acc = accel_correct / total
        brake_acc = brake_correct / total
        gear_acc = gear_correct / total
        clutch_acc = clutch_correct / total
        
        print(f"  Train Loss: {train_loss:.4f}, Accel Acc: {accel_acc:.4f}, Brake Acc: {brake_acc:.4f}, Gear Acc: {gear_acc:.4f}, Clutch Acc: {clutch_acc:.4f}")

        # Validation
        model.eval()
        val_loss = 0.0
        accel_correct = 0
        brake_correct = 0
        gear_correct = 0
        clutch_correct = 0
        total = 0
        
        with torch.no_grad():
            for features, steering_labels, accel_labels, brake_labels, gear_labels, clutch_labels in val_loader:
                features = features.to(device)
                steering_labels = steering_labels.to(device)
                accel_labels, brake_labels = accel_labels.to(device), brake_labels.to(device)
                gear_labels, clutch_labels = gear_labels.to(device), clutch_labels.to(device)
                
                steering_out, accel_out, brake_out, gear_out, clutch_out = model(features)
                
                loss_steering = criterion_steering(steering_out, steering_labels)
                loss_accel = criterion_accel(accel_out, accel_labels)
                loss_brake = criterion_brake(brake_out, brake_labels)
                loss_gear = criterion_gear(gear_out, gear_labels)
                loss_clutch = criterion_clutch(clutch_out, clutch_labels)
                loss = loss_steering + loss_accel + loss_brake + loss_gear + loss_clutch
                
                val_loss += loss.item() * features.size(0)
                accel_correct += (torch.argmax(accel_out, dim=1) == accel_labels).sum().item()
                brake_correct += (torch.argmax(brake_out, dim=1) == brake_labels).sum().item()
                gear_correct += (torch.argmax(gear_out, dim=1) == gear_labels).sum().item()
                clutch_correct += (torch.argmax(clutch_out, dim=1) == clutch_labels).sum().item()
                total += gear_labels.size(0)
        
        val_loss /= len(val_loader.dataset)
        val_accel_acc = accel_correct / total
        val_brake_acc = brake_correct / total
        val_gear_acc = gear_correct / total
        val_clutch_acc = clutch_correct / total
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        print(f"Epoch {epoch+1}/{num_epochs}:")
        
        print(f"  Val Loss: {val_loss:.4f}, Accel Acc: {val_accel_acc:.4f}, Brake Acc: {val_brake_acc:.4f}, Gear Acc: {val_gear_acc:.4f}, Clutch Acc: {val_clutch_acc:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pt')
            print("  Saved best model")
    
    # Plot loss curves
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig('loss_plot.png')
    plt.close()
    
    
    return train_losses, val_losses

In [ ]:
# Main execution
def main():

    num_accel_classes = len(train_dataset.data['Acceleration'].unique())  # Update: e.g., 2 for {0, 1}
    num_brake_classes = len(train_dataset.data['Braking'].unique())  # Update: e.g., 2 for {0, 1}
    num_gear_classes = len(train_dataset.data['Gear'].unique())  # Update: e.g., 8 for [-1, 0, 1, 2, 3, 4, 5, 6]
    num_clutch_classes = 2  # Binary {0, 1}
    
    # Initialize model
    model = TORCSNet(input_size, num_accel_classes, num_brake_classes, num_gear_classes, num_clutch_classes).to(device)
    
    # Loss functions
    criterion_steering = nn.MSELoss()
    
    # Compute class weights
    accel_counts = train_dataset.data['Acceleration'].value_counts()
    accel_class_weights = torch.tensor([1.0 / accel_counts.get(i, 1) for i in range(num_accel_classes)], dtype=torch.float32).to(device)
    brake_counts = train_dataset.data['Braking'].value_counts()
    brake_class_weights = torch.tensor([1.0 / brake_counts.get(i, 1) for i in range(num_brake_classes)], dtype=torch.float32).to(device)
    gear_counts = train_dataset.data['Gear'].value_counts()
    gear_class_weights = torch.tensor([1.0 / gear_counts.get(i, 1) for i in range(num_gear_classes)], dtype=torch.float32).to(device)
    clutch_counts = train_dataset.data['Clutch'].value_counts()
    clutch_class_weights = torch.tensor([1.0 / clutch_counts.get(i, 1) for i in [0, 1]], dtype=torch.float32).to(device)
    
    criterion_accel = nn.CrossEntropyLoss(weight=accel_class_weights)
    criterion_brake = nn.CrossEntropyLoss(weight=brake_class_weights)
    criterion_gear = nn.CrossEntropyLoss(weight=gear_class_weights)
    criterion_clutch = nn.CrossEntropyLoss(weight=clutch_class_weights)
    
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train model
    num_epochs = 50
    train_losses, val_losses = train_model(
        model, train_loader, val_loader,
        criterion_steering, criterion_accel, criterion_brake, criterion_gear, criterion_clutch,
        optimizer, num_epochs, device
    )
    
    # Evaluate on test set
    test_dataset = TORCSDataset('../data_generation/data/dirt-2/test_data.csv', sensor_cols, continuous_cols, discrete_cols)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)
    
    model.load_state_dict(torch.load('./best_model.pt'))
    model.eval()
    test_loss = 0.0
    accel_correct = 0
    brake_correct = 0
    gear_correct = 0
    clutch_correct = 0
    total = 0
    
    with torch.no_grad():
        for features, steering_labels, accel_labels, brake_labels, gear_labels, clutch_labels in test_loader:
            features = features.to(device)
            steering_labels = steering_labels.to(device)
            accel_labels, brake_labels = accel_labels.to(device), brake_labels.to(device)
            gear_labels, clutch_labels = gear_labels.to(device), clutch_labels.to(device)
            
            steering_out, accel_out, brake_out, gear_out, clutch_out = model(features)
            
            loss_steering = criterion_steering(steering_out, steering_labels)
            loss_accel = criterion_accel(accel_out, accel_labels)
            loss_brake = criterion_brake(brake_out, brake_labels)
            loss_gear = criterion_gear(gear_out, gear_labels)
            loss_clutch = criterion_clutch(clutch_out, clutch_labels)
            loss = loss_steering + loss_accel + loss_brake + loss_gear + loss_clutch
            
            test_loss += loss.item() * features.size(0)
            accel_correct += (torch.argmax(accel_out, dim=1) == accel_labels).sum().item()
            brake_correct += (torch.argmax(brake_out, dim=1) == brake_labels).sum().item()
            gear_correct += (torch.argmax(gear_out, dim=1) == gear_labels).sum().item()
            clutch_correct += (torch.argmax(clutch_out, dim=1) == clutch_labels).sum().item()
            total += gear_labels.size(0)
    
    test_loss /= len(test_loader.dataset)
    accel_acc = accel_correct / total
    brake_acc = brake_correct / total
    gear_acc = gear_correct / total
    clutch_acc = clutch_correct / total
    print(f"Test Loss: {test_loss:.4f}, Accel Acc: {accel_acc:.4f}, Brake Acc: {brake_acc:.4f}, Gear Acc: {gear_acc:.4f}, Clutch Acc: {clutch_acc:.4f}")


In [ ]:
if __name__ == "__main__":
    main()